# 3.13 丢弃法
除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法（dropout）[1] 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法（inverted dropout）

# 3.13.1 方法
回忆一下，3.8节（多层感知机）的图3.3描述了一个单隐藏层的多层感知机。其中输入个数为4，隐藏单元个数为5，且隐藏单元hi（i=1,…,5）的计算表达式为:

hi = ϕ(x1w1i + x2w2i + x3w3i + x4w4i + bi)

这里ϕ是激活函数，x1,…,x4是输入，隐藏单元i的权重参数为w1i,…,w4i，偏差参数为bi。当对该隐藏层使用丢弃法时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为p，那么有p的概率hi会被清零，有1−p的概率hi会除以1−p做拉伸。丢弃概率是丢弃法的超参数。

具体来说，设随机变量ξi为0和1的概率分别为p和1−p。使用丢弃法时我们计算新的隐藏单元h′i:

h′i = ξi / (1−p) * hi

由于E(ξi) = 1−p，因此

E(h′i) = E(ξi) / (1−p) * hi = hi

即丢弃法不改变其输入的期望值。让我们对图3.3中的隐藏层使用丢弃法，一种可能的结果如图3.5所示，其中h2和h5被清零。这时输出值的计算不再依赖h2和h5，在反向传播时，与这两个隐藏单元相关的权重的梯度均为0。由于在训练中隐藏层神经元的丢弃是随机的，即h1,…,h5都有可能被清零，输出层的计算无法过度依赖h1,…,h5中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。在测试模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法

# 3.13.2 从零开始实现
根据丢弃法的定义，我们可以很容易地实现它。下面的dropout函数将以drop_prob的概率丢弃X中的元素

In [18]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

# 这段代码实现了神经网络中的丢弃法(Dropout)正则化技术
# X: 输入张量，通常是神经网络的激活值
# drop_prob: 丢弃概率，每个元素被置零的概率
def dropout(X, drop_prob):
    X = X.float()

    # 确保丢弃概率在有效范围[0,1]内
    # 若不满足，程序将抛出AssertionError
    assert 0 <= drop_prob <= 1

    # 计算元素被保留的概率，与丢弃概率互补
    keep_prob = 1 - drop_prob
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        # 特殊情况：如果保留概率为0(丢弃概率为1)
        # 直接返回与X形状相同的全零张量
        return torch.zeros_like(X)

    # torch.rand(X.shape): 生成与X相同形状的[0,1]均匀分布随机张量(每个元素是从[0,1)区间内均匀采样的随机浮点数)
    # < keep_prob: 创建布尔掩码，概率为keep_prob的元素为True
    # 每个随机值都与keep_prob标量进行比较(比较操作独立应用于张量的每个元素)
    # 结果是一个与输入同形状的布尔张量——
    # 对于 random_tensor 中的每个元素:
    # 如果 元素值 < keep_prob，对应位置为 True, 否则为 False
    # .float(): 将布尔值转换为浮点数(0.0或1.0)
    mask = (torch.rand(X.shape) < keep_prob).float()

    #mask * X: 元素乘法，保留掩码中值为1的元素，其余置0
    # / keep_prob: 重要的缩放步骤，使输出期望值不变
    # 这种缩放称为"反向丢弃"(inverted dropout)
    return mask * X / keep_prob

我们运行几个例子来测试一下dropout函数。其中丢弃概率分别为0、0.5和1

In [19]:
X = torch.arange(16).view(2, 8)
dropout(X, 0)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [20]:
dropout(X, 0.5)

tensor([[ 0.,  2.,  0.,  0.,  8.,  0., 12.,  0.],
        [16.,  0.,  0.,  0.,  0., 26.,  0.,  0.]])

In [21]:
dropout(X, 1.0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

# 3.13.2.1 定义模型参数
实验中，我们依然使用3.6节（softmax回归的从零开始实现）中介绍的Fashion-MNIST数据集。我们将定义一个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数都是256

In [22]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

# np.random.normal 函数用于从正态分布(高斯分布)中抽取随机样本，它是神经网络权重初始化的常用方法之一
W1 = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs, num_hiddens1)), dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens1, requires_grad=True)
W2 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens1, num_hiddens2)), dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_hiddens2, requires_grad=True)
W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2, num_outputs)), dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs, requires_grad=True)

params = [W1, b1, W2, b2, W3, b3]

# 3.13.2.2 定义模型
下面定义的模型将全连接层和激活函数ReLU串起来，并对每个激活函数的输出使用丢弃法。我们可以分别设置各个层的丢弃概率。通常的建议是把靠近输入层的丢弃概率设得小一点。在这个实验中，我们把第一个隐藏层的丢弃概率设为0.2，把第二个隐藏层的丢弃概率设为0.5。我们可以通过参数is_training来判断运行模式为训练还是测试，并只需在训练模式下使用丢弃法

In [23]:
drop_prob1, drop_prob2 = 0.2, 0.5

def net(X, is_training=True):
    X = X.view(-1, num_inputs)
    H1 = (torch.matmul(X, W1) + b1).relu()
    if is_training:  # 只在训练模型时使用丢弃法
        H1 = dropout(H1, drop_prob1)  # 在第一层全连接后添加丢弃层
    H2 = (torch.matmul(H1, W2) + b2).relu()
    if is_training:
        H2 = dropout(H2, drop_prob2)  # 在第二层全连接后添加丢弃层
    return torch.matmul(H2, W3) + b3

我们在对模型评估的时候不应该进行丢弃，所以我们修改一下d2lzh_pytorch中的evaluate_accuracy函数

In [24]:
# 本函数已保存在d2lzh_pytorch
# 定义一个函数，用于评估模型在数据集上的准确率
# data_iter: 数据迭代器，用于批量获取测试数据
# net: 要评估的神经网络模型
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0   # acc_sum: 初始化正确预测的样本数量为0; n: 初始化评估的样本总数为0
    for X, y in data_iter:
        if isinstance(net, torch.nn.Module):   # 检查模型是否是PyTorch的标准模型(nn.Module的实例)
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train() # 改回训练模式
        else: # 自定义的模型
            if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                # 将is_training设置成False
                acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
            else:
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
        n += y.shape[0]
    return acc_sum / n

注：将上述evaluate_accuracy写回d2lzh_pytorch后要重启一下jupyter kernel才会生效

# 3.13.2.3 训练和测试模型
这部分与之前多层感知机的训练和测试类似

In [25]:
num_epochs, lr, batch_size = 5, 100.0, 256
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

epoch 1, loss 0.0045, train acc 0.554, test acc 0.705
epoch 2, loss 0.0023, train acc 0.785, test acc 0.756
epoch 3, loss 0.0019, train acc 0.822, test acc 0.793
epoch 4, loss 0.0017, train acc 0.839, test acc 0.846
epoch 5, loss 0.0016, train acc 0.849, test acc 0.830


注：这里的学习率设置的很大，原因同3.9.6节

# 3.13.3 简洁实现
在PyTorch中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，Dropout层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时（即model.eval()后），Dropout层并不发挥作用

In [26]:
# drop_prob1, drop_prob2 = 0.2, 0.5
# 这段代码创建了一个带有丢弃法正则化的多层神经网络
net = nn.Sequential(
        # d2l.FlattenLayer(): 自定义层，将输入的多维张量展平为二维
        # 例如：将 (batch_size, 1, 28, 28) 的图像转换为 (batch_size, 784)
        d2l.FlattenLayer(),   
        nn.Linear(num_inputs, num_hiddens1),  
        nn.ReLU(),   # 激活函数，引入非线性变换
        nn.Dropout(drop_prob1),   # 第一个丢弃层，随机断开第一隐藏层的部分连接
        nn.Linear(num_hiddens1, num_hiddens2), 
        nn.ReLU(),
        nn.Dropout(drop_prob2),
        nn.Linear(num_hiddens2, num_outputs)   # 输出维度为num_outputs = 10，对应Fashion-MNIST的10个类别
        )

for param in net.parameters():
    # 使用正态分布初始化网络参数,下划线表示这是一个原地操作，直接修改参数
    nn.init.normal_(param, mean=0, std=0.01)   

下面训练并测试模型

In [27]:
optimizer = torch.optim.SGD(net.parameters(), lr = 0.5)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0046, train acc 0.552, test acc 0.740
epoch 2, loss 0.0023, train acc 0.785, test acc 0.808
epoch 3, loss 0.0019, train acc 0.819, test acc 0.747
epoch 4, loss 0.0018, train acc 0.836, test acc 0.810
epoch 5, loss 0.0016, train acc 0.847, test acc 0.790


注：由于这里使用的是PyTorch的SGD而不是d2lzh_pytorch里面的sgd，所以就不存在3.9.6节那样学习率看起来很大的问题了。

# 小结
我们可以通过使用丢弃法应对过拟合。

丢弃法只在训练模型时使用